<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/fastfouriertransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math
import cmath # For complex math if needed, though `complex` type handles most

class FastFourierTransformTests(unittest.TestCase):

    def test_simple_fft(self): # Renamed from testSimple
        print("Testing complex direct FFT...")
        # typedef std::complex<Real> cx;
        # cx a[] = { cx(0,0), cx(1,1), cx(3,3), cx(4,4),
        #            cx(4,4), cx(3,3), cx(1,1), cx(0,0) };
        # Input array `a` (Python list of complex numbers)
        a_input = [complex(0,0), complex(1,1), complex(3,3), complex(4,4),
                   complex(4,4), complex(3,3), complex(1,1), complex(0,0)]

        # cx b[8]; // Output array
        # FastFourierTransform fft(3); // order = 3 => 2^3 = 8 points
        fft = ql.FastFourierTransform(3) # order = 3

        # fft.transform(a, a+8, b);
        # The Python transform method likely takes the input list and returns the output list
        # Or it might take input and output lists as arguments.
        # Let's assume it takes input and returns output (common SWIG pattern for fixed size output).
        # If it's in-place or needs pre-allocated output, adjust accordingly.
        # QL Python's FFT.transform takes a list of complex numbers.
        b_output = fft.transform(a_input)

        expected = [complex(16,16), complex(-4.828427, -11.656854), # More precise from numpy
                    complex(0,0),   complex(-0.343146, 0.828427),
                    complex(0,0),   complex(0.828427, -0.343146),
                    complex(0,0),   complex(-11.656854, -4.828427)]

        self.assertEqual(len(b_output), len(expected))
        for i in range(len(b_output)):
            # Using assertAlmostEqual for complex numbers might need custom logic or checking real/imag separately
            self.assertAlmostEqual(b_output[i].real, expected[i].real, places=2, # C++ used 1.0e-2
                                   msg=f"FFT real part mismatch at index {i}: "
                                       f"Calc: {b_output[i]}, Exp: {expected[i]}")
            self.assertAlmostEqual(b_output[i].imag, expected[i].imag, places=2,
                                   msg=f"FFT imag part mismatch at index {i}: "
                                       f"Calc: {b_output[i]}, Exp: {expected[i]}")

    def test_inverse_fft_convolution(self): # Renamed from testInverse
        print("Testing convolution via inverse FFT...")
        # Array x(3); x[0] = 1; x[1] = 2; x[2] = 3;
        x_input_list = [1.0, 2.0, 3.0]
        # ql.Array can be created from a list
        # x_ql_array = ql.Array(x_input_list) # If needed by inverse_transform

        # size_t order = FastFourierTransform::min_order(x.size())+1;
        # FastFourierTransform fft(order);
        # size_t nFrq = fft.output_size();
        # The C++ `inverse_transform` takes iterators. QL Python's `inverse_transform`
        # likely takes a list. The input `x` is real.
        # The FFT length `nFrq` needs to be >= 2*N-1 for convolution of N-length sequence with itself.
        # Here N = 3. 2*3-1 = 5. min_order for 5 is 3 (2^3=8).
        # C++: order = min_order(x.size()) + 1 = min_order(3) + 1.
        # min_order(3) is 2 (2^2=4 is >= 3). So order = 2+1=3. nFrq = 2^3 = 8.
        order = ql.FastFourierTransform.get_min_order(len(x_input_list)) + 1
        fft = ql.FastFourierTransform(order)
        n_freq = fft.output_size() # Should be 8

        # std::vector< std::complex<Real> > ft (nFrq);
        # fft.inverse_transform(x.begin(), x.end(), ft.begin());
        # inverse_transform on real input (padded with zeros to n_freq)
        # The QL Python `inverse_transform` might expect a complex list as input.
        # If it takes real input and outputs complex, or if it has an overload.
        # Let's assume it takes a real list (padded) and outputs complex.
        # Or, it might take a complex list where imag parts are zero.
        x_input_padded_real = x_input_list + [0.0] * (n_freq - len(x_input_list))

        # QL FFT `inverse_transform` typically takes complex input.
        # To use it with real data `x_input_list`, we treat `x` as the real part of a complex sequence.
        # The C++ `inverse_transform(x.begin(), x.end(), ft.begin())` where x is Real Array
        # suggests an overload or that the template handles Real*.
        # Python binding for `inverse_transform` likely expects list of complex.
        x_input_complex_padded = [complex(val, 0.0) for val in x_input_padded_real]
        ft_output = fft.inverse_transform(x_input_complex_padded)

        # Calculate power spectrum: tmp[i] = norm(ft[i])
        # std::norm(z) = z.real()^2 + z.imag()^2
        tmp_power_spectrum_real = [(c.real**2 + c.imag**2) for c in ft_output]

        # Second inverse FFT on the power spectrum (which is real)
        # Pad tmp_power_spectrum_real with zeros if its length is less than n_freq (shouldn't be)
        # Treat tmp_power_spectrum_real as real part of complex input for IFFT
        tmp_power_spectrum_complex = [complex(val, 0.0) for val in tmp_power_spectrum_real]
        ft_convolution_output = fft.inverse_transform(tmp_power_spectrum_complex)

        # Check convolution results
        # result[k] / n_freq corresponds to sum_{j} x[j]*x[j-k] (auto-correlation like)
        # For positive k, it's sum_{j} x[j]*x[j+k] if we consider x to be defined outside its original range with zeros.

        # Term 0: sum(x[i]^2)
        calculated0 = ft_convolution_output[0].real / n_freq
        expected0 = x_input_list[0]**2 + x_input_list[1]**2 + x_input_list[2]**2
        self.assertAlmostEqual(calculated0, expected0, places=10,
                               msg=f"Convolution(0): Calc {calculated0}, Exp {expected0}")

        # Term 1: x[0]*x[1] + x[1]*x[2]
        calculated1 = ft_convolution_output[1].real / n_freq
        expected1 = x_input_list[0]*x_input_list[1] + x_input_list[1]*x_input_list[2]
        self.assertAlmostEqual(calculated1, expected1, places=10,
                               msg=f"Convolution(1): Calc {calculated1}, Exp {expected1}")

        # Term 2: x[0]*x[2]
        calculated2 = ft_convolution_output[2].real / n_freq
        expected2 = x_input_list[0]*x_input_list[2]
        self.assertAlmostEqual(calculated2, expected2, places=10,
                               msg=f"Convolution(2): Calc {calculated2}, Exp {expected2}")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

Input/Output of transform and inverse_transform: The C++ methods use iterators. SWIG often wraps these to take Python lists or specific QuantLib vector types (like ql.RealVector, ql.ComplexVector).
The transform method in ql.FastFourierTransform indeed takes a list of Python complex numbers and returns a list of Python complex numbers.
The inverse_transform method also takes a list of Python complex numbers and returns a list of Python complex numbers. When the conceptual input is real (like x or tmp in the C++ test), it must be passed as a list of complex numbers where the imaginary parts are zero.
Padding: FFT algorithms operate on sequences of length 2^order. If the input sequence is shorter, it's typically padded with zeros (or the transform handles this internally up to output_size()). The C++ test's inverse_transform(x.begin(), x.end(), ...) implies that only the first x.size() elements are non-zero, and the rest up to nFrq are treated as zero for the transform. This is handled by creating x_input_complex_padded in the Python version.
Normalization: Standard FFT algorithms might differ in their normalization factors for forward and inverse transforms. QuantLib's FastFourierTransform class encapsulates this. The scaling by 1/nFrq in the convolution test is part of the mathematical property of using FFT for convolution.